In [1]:
%%script C:\Users\Jan Catherine\AppData\Local\Programs\Python\Python38\python.exe
import sys
print(sys.version)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [2]:
pwd

'C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN'

In [3]:
import os
os.chdir('../../Data_Collection_02/02_Preprocessing/05_CNN')

In [4]:
#pip install tensorflow

In [5]:
#importing libraries
"""
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
"""

'\nfrom keras.models import Sequential\nfrom keras.layers import Dense, Dropout, Activation, Flatten\nfrom keras.layers import Convolution2D, MaxPooling2D\nfrom sklearn.model_selection import train_test_split\nimport pandas as pd\nimport numpy as np\n'

In [6]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy


from sklearn.model_selection import KFold

#import pydot

In [7]:
dataset = pd.read_csv('01_URL_Train.csv', encoding='latin-1')
testdata = pd.read_csv('01_URL_Test.csv', encoding='latin-1')

train_data = dataset
test_data = testdata

#Oversampling
count_class_0, count_class_1 = train_data.Fake.value_counts()

df_class_0 = train_data[train_data['Fake'] == 0]
df_class_1 = train_data[train_data['Fake'] == 1]

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

df_test_over = df_test_over.reset_index(drop=True)
print('Random over-sampling:')
print(df_test_over.Fake.value_counts())

train_data = df_test_over

print(train_data.Fake.value_counts())

train_data.dropna(axis=0, how ='any', inplace=True)
train_data['Num_words_url'] = train_data['URL'].apply(lambda x:len(str(x)))

print('-------Train data--------')
print(train_data['Fake'].value_counts())
print(len(train_data))
print('-------------------------')

max_train_url_length  = train_data['Num_words_url'].max()



test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_url'] = test_data['URL'].apply(lambda x:len(str(x))) 

max_test_url_length  = test_data['Num_words_url'].max()

print('-------Test data--------')
print(test_data['Fake'].value_counts())
print(len(test_data))
print('-------------------------')

print('Train Max Length :'+str(max_train_url_length))
print('Test Max Sentence Length :'+str(max_test_url_length))


Random over-sampling:
0    5760
1    5760
Name: Fake, dtype: int64
0    5760
1    5760
Name: Fake, dtype: int64
-------Train data--------
0    5760
1    5760
Name: Fake, dtype: int64
11520
-------------------------
-------Test data--------
0    1429
1     184
Name: Fake, dtype: int64
1613
-------------------------
Train Max Length :255
Test Max Sentence Length :250


In [8]:
train_data.head()

,URL,Fake,Num_words_url
0,https://www.thehindu.com/opinion/lead/a-key-ar...,0,102
1,https://www.mckinsey.com/industries/financial-...,0,150
2,https://coronadotimes.com/news/2020/06/14/nail...,0,107
3,https://aricjournal.biomedcentral.com/articles...,0,73
4,https://www.idealista.com/en/news/property-for...,0,144


In [9]:
num_words = 12000


URLchar = Tokenizer(num_words=num_words, oov_token="unk", char_level=True)
URLchar.fit_on_texts(train_data['URL'].tolist())

print(str(URLchar.texts_to_sequences(['5G Global bill airfare'])))

[[34, 24, 1, 24, 15, 6, 25, 8, 15, 1, 25, 9, 15, 15, 1, 8, 9, 11, 28, 8, 11, 3]]


In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data['URL'].tolist(),\
                                                      train_data['Fake'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['Fake'].tolist(),\
                                                      random_state=0)

print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(y_valid)))

x_train = np.array( URLchar.texts_to_sequences(X_train) )
x_valid = np.array( URLchar.texts_to_sequences(X_valid) )
x_test  = np.array( URLchar.texts_to_sequences(test_data['URL'].tolist()) )

x_train = pad_sequences(x_train, padding='post', maxlen=260)
x_valid = pad_sequences(x_valid, padding='post', maxlen=260)
x_test = pad_sequences(x_test, padding='post', maxlen=260)

print(x_train[0])

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels, 2))
#print(train_labels)
valid_labels = le.transform(y_valid)
valid_labels = np.asarray( tf.keras.utils.to_categorical(valid_labels, 2))


test_labels = le.transform(test_data['Fake'].tolist())
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels, 2))
list(le.classes_)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

Train data len:9216
Class distributionCounter({0: 4608, 1: 4608})
Valid data len:2304
Class distributionCounter({1: 1152, 0: 1152})


C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_11248\1508549292.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array( URLchar.texts_to_sequences(X_train) )
C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_11248\1508549292.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array( URLchar.texts_to_sequences(X_valid) )


[14  4  4 16  5 27  7  7 13 13 13 17 10  3 13  5 26 35 17 12  6 19  7 10
  3 13  5  7  9 10 20  9  8  7 10  3 13  5  2  6 28  2 12 14  9 10  3  5
  3  2 15  8 25  2 12 11  3  8  4  9 10 24  2  8 10  4 14 11  8 39  2 15
  9 30  3  2 16  8  4 14  6 24  3 10  5  2  9 10  2 16  8 30  9  5  4  8
 10  2 28  8 30  3  2  5  8 29  5  2  9  4  5  2 28  6 11  3  9 24 10  2
  6 28 28  9 12  3  2 23 37 32 36 26 23 31 17 14  4 19 15  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_11248\1508549292.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test  = np.array( URLchar.texts_to_sequences(test_data['URL'].tolist()) )


In [11]:
test_data.head()

,URL,Fake,Num_words_url
0,https://bhekisisa.org/article/2020-06-03-coron...,0,139
1,https://www.express.co.uk/travel/articles/1246...,0,132
2,https://www.thesun.co.uk/tvandshowbiz/11851914...,0,87
3,https://www.inquirer.com/opinion/china-reparat...,0,98
4,https://scdhec.gov/covid19/covid-19-testing-lo...,0,53


In [12]:
count =0
print('======Train dataset ====')
for value,label in train_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count =0
print('======Validation dataset ====')
for value,label in valid_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count = 0
print('======Test dataset ====')
for value,label in test_ds:
    count += 1
    print(value,label)
    if count==3:
        break  


======Train dataset ====
tf.Tensor(
[14  4  4 16  5 27  7  7 13 13 13 17 10  3 13  5 26 35 17 12  6 19  7 10
  3 13  5  7  9 10 20  9  8  7 10  3 13  5  2  6 28  2 12 14  9 10  3  5
  3  2 15  8 25  2 12 11  3  8  4  9 10 24  2  8 10  4 14 11  8 39  2 15
  9 30  3  2 16  8  4 14  6 24  3 10  5  2  9 10  2 16  8 30  9  5  4  8
 10  2 28  8 30  3  2  5  8 29  5  2  9  4  5  2 28  6 11  3  9 24 10  2
  6 28 28  9 12  3  2 23 37 32 36 26 23 31 17 14  4 19 15  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0], shape=(260,), dtype=int32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor(
[14  4  4 16  5 27  7  7 24  9 40 19  6 20  6 17 12  6 19 17  8 18  7 23
 21 23 21  7 2

In [13]:
#CNN MODEL

max_features = 11718
embedding_dim = 64 #same as URLNet
sequence_length = 260

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

inputs = np.concatenate((x_train,x_valid), axis=0)
targets = np.concatenate((train_labels, valid_labels), axis=0)

#regularizer prevents overfitting
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                        embeddings_regularizer = regularizers.l2(0.0005)))                                    

    model.add(tf.keras.layers.Conv1D(128,3, activation='relu',\
                                     kernel_regularizer = regularizers.l2(0.0005),\
                                     bias_regularizer = regularizers.l2(0.0005)))                               


    model.add(tf.keras.layers.GlobalMaxPooling1D())

    model.add(tf.keras.layers.Dropout(0.5)) #to reduce overfitting

    #final classification, 2 classes
    model.add(tf.keras.layers.Dense(2, activation='sigmoid',\
                                    kernel_regularizer=regularizers.l2(0.001),\
                                    bias_regularizer=regularizers.l2(0.001),))




    model.summary()
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer='Nadam', metrics=["CategoricalAccuracy"])
    
    epochs = 100
    # Fit the model using the train and test datasets.
    #history = model.fit(x_train, train_labels,validation_data= (x_test,test_labels),epochs=epochs )
    history = model.fit(inputs[train], targets[train],
                        epochs= epochs ,
                        verbose=1)
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 260, 64)           750016    
                                                                 
 conv1d (Conv1D)             (None, 258, 128)          24704     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 774978 (2.96 MB)
Trainable params: 774978 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [14]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.3239861726760864 - Accuracy: 95.87673544883728%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.32738861441612244 - Accuracy: 96.26736044883728%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.331327885389328 - Accuracy: 95.78993320465088%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.3255639970302582 - Accuracy: 95.61631679534912%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.3301909565925598 - Accuracy: 95.92013955116272%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 95.89409708976746 (+- 0.21369004587257545)
> Loss: 0.32769152522087097
--------------------------

In [15]:
history.history

{'loss': [0.7003568410873413,
  0.6003673076629639,
  0.550531268119812,
  0.5197961330413818,
  0.4954182207584381,
  0.47076544165611267,
  0.46447882056236267,
  0.4491487145423889,
  0.4356113076210022,
  0.43428078293800354,
  0.4208761155605316,
  0.4178127944469452,
  0.41123145818710327,
  0.411169171333313,
  0.40106093883514404,
  0.39885255694389343,
  0.3941323459148407,
  0.3850920796394348,
  0.3877650499343872,
  0.3910851776599884,
  0.38574928045272827,
  0.3789204955101013,
  0.3833741843700409,
  0.3832370340824127,
  0.37476783990859985,
  0.37751320004463196,
  0.37096333503723145,
  0.3762844502925873,
  0.3733479082584381,
  0.37590867280960083,
  0.36770114302635193,
  0.36843541264533997,
  0.37148064374923706,
  0.36464905738830566,
  0.3642212450504303,
  0.3658428490161896,
  0.3681264817714691,
  0.36624762415885925,
  0.36289942264556885,
  0.3623206317424774,
  0.36215126514434814,
  0.36536750197410583,
  0.36544859409332275,
  0.3641110360622406,
  0.36

In [16]:
"""
plt.plot(history.history['loss'], label=' training data')
plt.plot(history.history['val_loss'], label='validation data')
plt.title('Loss for Text Classification')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

'\nplt.plot(history.history[\'loss\'], label=\' training data\')\nplt.plot(history.history[\'val_loss\'], label=\'validation data\')\nplt.title(\'Loss for Text Classification\')\nplt.ylabel(\'Loss value\')\nplt.xlabel(\'No. epoch\')\nplt.legend(loc="upper left")\nplt.show()\n'

In [17]:
"""
plt.plot(history.history['categorical_accuracy'], label=' (training data)')
plt.plot(history.history['val_categorical_accuracy'], label='CategoricalCrossentropy (validation data)')
plt.title('CategoricalAccuracy for Text Classification')
plt.ylabel('CategoricalAccuracy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

'\nplt.plot(history.history[\'categorical_accuracy\'], label=\' (training data)\')\nplt.plot(history.history[\'val_categorical_accuracy\'], label=\'CategoricalCrossentropy (validation data)\')\nplt.title(\'CategoricalAccuracy for Text Classification\')\nplt.ylabel(\'CategoricalAccuracy value\')\nplt.xlabel(\'No. epoch\')\nplt.legend(loc="upper left")\nplt.show()\n'

In [18]:
model.save('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLchar') 
json_string = URLchar.to_json()

INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\URLchar\assets


INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\URLchar\assets


In [19]:
import json
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLchar\\URLchar.json', 'w') as outfile:
    json.dump(json_string, outfile)

In [20]:
#reload model

new_model = tf.keras.models.load_model('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLchar')
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 260, 64)           750016    
                                                                 
 conv1d_4 (Conv1D)           (None, 258, 128)          24704     
                                                                 
 global_max_pooling1d_4 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 2)                 258       
                                                                 
Total params: 774978 (2.96 MB)
Trainable params: 774978 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [21]:
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\URLchar\\URLchar.json') as json_file:
    json_string = json.load(json_file)
tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string)

In [22]:
x_test  = np.array( tokenizer1.texts_to_sequences(test_data['URL'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=260)

C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_11248\2077557873.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test  = np.array( tokenizer1.texts_to_sequences(test_data['URL'].tolist()) )


In [23]:
# Generate predictions (probabilities -- the output of the last layer)
# on test  data using `predict`
print("Generate predictions for all samples")
predictions = new_model.predict(x_test)
print(predictions)
predict_results = predictions.argmax(axis=1)

Generate predictions for all samples
51/51 [==============================] - 0s 6ms/step
[[0.99622256 0.00377741]
 [0.9815708  0.01842912]
 [0.7696982  0.23030218]
 ...
 [0.8019059  0.19809413]
 [0.9904026  0.00959739]
 [0.9476664  0.05233338]]


In [24]:
test_data['pred_fake']= predict_results
test_data['pred_fake'] = np.where((test_data.pred_fake == '0'),0,test_data.pred_fake)
test_data['pred_fake'] = np.where((test_data.pred_fake == '1'),1,test_data.pred_fake)

In [25]:
print(type(test_data['Fake']))
test_data['Fake'].head()


<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    0
4    0
Name: Fake, dtype: int64

In [26]:
print(type(test_data['pred_fake']))
test_data['pred_fake'].head()

<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    1
4    0
Name: pred_fake, dtype: int64

In [27]:
labels = [0, 1]
    
print(classification_report(test_data['Fake'].tolist(),test_data['pred_fake'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1429
           1       0.73      0.73      0.73       184

    accuracy                           0.94      1613
   macro avg       0.85      0.85      0.85      1613
weighted avg       0.94      0.94      0.94      1613



In [28]:
"""precisionscore = precision_score(y_test, new_model.predict(x_test))
accuracyscore = accuracy_score(y_test, new_model.predict(x_test))
recallscore = recall_score(y_test, new_model.predict(x_test))
f1score = f1_score(y_test, new_model.predict(x_test))
cm = confusion_matrix(y_test, new_model.predict(x_test))

print(precisionscore, accuracyscore, recallscore, f1score, cm)"""

'precisionscore = precision_score(y_test, new_model.predict(x_test))\naccuracyscore = accuracy_score(y_test, new_model.predict(x_test))\nrecallscore = recall_score(y_test, new_model.predict(x_test))\nf1score = f1_score(y_test, new_model.predict(x_test))\ncm = confusion_matrix(y_test, new_model.predict(x_test))\n\nprint(precisionscore, accuracyscore, recallscore, f1score, cm)'